""" @author: faradars """

'''  Pre_Processing Codes Include:
    
    - One_Hot Encoding
    - Image Re_Sizing
    - Image Augmentation
    - Image Re_Color
    - Image Saving
    - Show Results


In [1]:
!pip install torch
!pip install torchvision

In [2]:
!pip install scikit-plot

In [3]:
!pip install opencv-python

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import math
import time
import os
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchsummary import summary
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import itertools
from sklearn.metrics import confusion_matrix
import cv2
from sklearn.model_selection import cross_val_score
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import scikitplot as skplt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
np.set_printoptions(precision=2)
use_gpu = torch.cuda.is_available()
np.random.seed(1234)

In [5]:
# 1. Import Required Modules

import os
import sys
import glob
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from skimage.transform import resize
from skimage.io import imread, imshow, imsave
from skimage.viewer import ImageViewer
#from OOP import Pre_Processing_R2g


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Viewer requires Qt
  


In [6]:
from google.colab import drive

drive.mount('/content/drive')

!ls 'drive/My Drive'

Mounted at /content/drive
'Abstract.Samira Asadzadeh.pdf'
'Argent Wallet'
'Colab Notebooks'
'FINAL DOCUMENT .docx'
'FINAL DOCUMENT .pdf'
'Final PEdit.pptx'
 form1.doc.gdoc
 Fusion_of_CNN_Features_and_Ensemble_Classifiers.ipynb
"I am sharing '۱' with you"
 IMG-1109.PNG
 Project
 Sample
 SAVABEGH.pdf
'Thesis(Summer 2019).pdf'
 Untitled0.ipynb
'zaban nosrat mobile.rar'
 zangkhor.mp3
'آدرس کیف پول.gdoc'


In [7]:
# 2. Set Image Path

IMAGE_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/inpouts/'
IMAGE_MASK_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/mask/'


In [9]:
# 3. Get Images and Set Some Parameters, One_Hot Encoding Technique

IMG_HEIGHT = 720
IMG_WIDTH = 1128
IMG_CHANNELS = 3

In [12]:
#image4 = mpimg.imread('/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/mask/1_268_10_3.jpg')
IMG_Dataset = next(os.walk(IMAGE_PATH))[2] 

Inputs = np.zeros((len(IMG_Dataset), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                  dtype = np.uint8)

print('Loading Images & Masks, Please Wait')
sys.stdout.flush()
for n, f in tqdm(enumerate(IMG_Dataset), total = len(IMG_Dataset)): #n شماره فال و f نام فایل  ذخیره میگردد
    Imagesmask = imread (IMAGE_MASK_PATH + f.replace('0.', ''))[:,:,:IMG_CHANNELS]
    Inputs[n] = Imagesmask

    filename_w_ext = os.path.basename(f)
    filename, file_extension = os.path.splitext(filename_w_ext)
    
    #img = mpimg.imread('/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/mask/1_268_10_3.jpg')
    plt.imshow(Inputs[n]), plt.title('Orginnal Masks')
    plt.show()
    median = cv2.medianBlur(Inputs[n],5)
    #تبدیل فضای رنگ bgr to hsv
    hsv = cv2.cvtColor(median,cv2.COLOR_BGR2HSV )
    plt.imshow(hsv), plt.title('Hsv Masks')
    plt.show() 
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([120,255,255])
 
    lower_green = np.array([50,50,50])
    upper_green = np.array([60,255,255])
 
    lower_red = np.array([0,50,50])
    upper_red = np.array([0,255,255])
    #آستانه گیری(cv2.inRange())
    maskb = cv2.inRange(hsv, lower_blue, upper_blue)
    maskg = cv2.inRange(hsv, lower_green, upper_green)
    maskr = cv2.inRange(hsv, lower_red, upper_red)
    output_path = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Masks_Train'
    cv2.imwrite(os.path.join(output_path, filename +'.jpg' ),maskb)
    # and تصویر اصلی با ماسک
    #میشود (cv2.bitwise_and())
    resb = cv2.bitwise_and(Inputs[n],Inputs[n], mask= maskb)
    resg = cv2.bitwise_and(Inputs[n],Inputs[n], mask= maskg)
    resr = cv2.bitwise_and(Inputs[n],Inputs[n], mask= maskr)
    plt.figure(figsize=(14, 14))
    plt.subplot(231), plt.imshow(maskb, cmap='gray')
    plt.title('b'), plt.xticks([]), plt.yticks([])
    plt.subplot(232), plt.imshow(maskg)
    plt.title('g'), plt.xticks([]), plt.yticks([])
    plt.subplot(233), plt.imshow(maskr)
    plt.title('r'), plt.xticks([]), plt.yticks([])
    plt.show()
    plt.figure(figsize=(14, 14)) 
    plt.subplot(231), plt.imshow(resb, cmap='gray')
    plt.title('resb'), plt.xticks([]), plt.yticks([])
    plt.subplot(232), plt.imshow(resg)
    plt.title('resg'), plt.xticks([]), plt.yticks([])
    plt.subplot(233), plt.imshow(resr)
    plt.title('resr'), plt.xticks([]), plt.yticks([])
    plt.show()

cv2.waitKey()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [13]:
Ground_Truth_PATH = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Masks_Train/'
IMG_Dataset = next(os.walk(Ground_Truth_PATH))[2] 

#Inputs = np.zeros((len(IMG_Dataset), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                  #dtype = np.uint8)
Ground_Truth = np.zeros((len(IMG_Dataset), IMG_HEIGHT, IMG_WIDTH, 2),
                  dtype = np.bool)
print('Loading Images & Masks, Please Wait')
sys.stdout.flush()

for n, g in tqdm(enumerate(IMG_Dataset), total = len(IMG_Dataset)): #n شماره فال و f نام فایل  ذخیره میگردد
    Images = imread (Ground_Truth_PATH + g)
    #Inputs[n] = Images
    filename_w_ext = os.path.basename(g)
    filename1, file_extension = os.path.splitext(filename_w_ext)
    plt.imshow(Images ), plt.title('Orginnal Masks')
    plt.show()
    
    Masks = imread(Ground_Truth_PATH + g)              
    Masks = np.squeeze(Masks).astype(np.bool)     
    Ground_Truth[n, :, :, 0] = ~Masks
    Ground_Truth[n, :, :, 1] = Masks
    Masks = np.array(255*np.squeeze(Masks), np.uint8)
    #Masks =  np.array(Masks).astype(np.float32)
    plt.imshow(Masks, cmap='gray'), plt.title('Ground_Truth')
    plt.show()
    Masks = cv2.resize(Masks  , (224, 224))
    output_path = '/content/drive/MyDrive/Project/BreastCancer/Dataset/Dataset/Masks_Train/onehotencoding'
    cv2.imwrite(os.path.join(output_path, filename1 +'.jpg' ), Masks)
print('Loading Images and Masks Completed Successfully!')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 4. Define Functions: Re_Size, Augmentation, Re_Color, Im_Saving
# 4.1. Function_1: Pre_Process_Re_Size():

#IMG_HEIGHT_RESIZED = 255
#IMG_WIDTH_RESIZED = 255
#IMG_CHANNELS_RESIZED = 3
#
#def Pre_Process_Re_Size(imgs):
#    
#    img_p = np.zeros((imgs.shape[0], IMG_HEIGHT_RESIZED, IMG_WIDTH_RESIZED
#                      , IMG_CHANNELS_RESIZED), dtype = np.uint8)
#    
#    for i in range(imgs.shape[0]):
#        
#       img_p[i] = resize(imgs[i], (IMG_HEIGHT_RESIZED, IMG_WIDTH_RESIZED,
#               IMG_CHANNELS_RESIZED), preserve_range=True)
#       
#    return img_p
#
#''' ------------------------------------------------------- '''
#
## 4.2. Function_2: Pre_Process_Augmentation():
#
#IMAGE_INITIAL_PATH = '/home/faradars/Desktop/Data/Augmentation/' 
#IMAGE_AUGMENTATED_PATH = '/home/faradars/Desktop/Data/Rotated/' 
#
#Data_Gen = image.ImageDataGenerator(rotation_range=30)
#IMG_AUG = Data_Gen.flow_from_directory(IMAGE_INITIAL_PATH, batch_size=1
#                                       , save_to_dir=IMAGE_AUGMENTATED_PATH,
#                                       save_prefix='Aug', target_size=(768,896))
#
#for i in range(9):
#    IMG_AUG.next()
#    
#def Pre_Process_Augmentation(Path_Images):
#    
#    Images_List = glob.glob(Path_Images)
#    Figure = plt.figure()
#    
#    for i in range (9):
#        Images_A = Image.open(Images_List[i])
#        Sub_Image_Show = Figure.add_subplot(331 + i)
#        Sub_Image_Show.imshow(Images_A)
#    plt.show()
#    return Figure
#    
#Image_Original = Pre_Process_Augmentation(IMAGE_INITIAL_PATH + 'input/*')
#Image_Original.savefig(IMAGE_AUGMENTATED_PATH + '/Original.png', dpi = 200,
#                       papertype = 'a5')
#
#Image_Augmentation = Pre_Process_Augmentation(IMAGE_AUGMENTATED_PATH + '/*')
#Image_Augmentation.savefig(IMAGE_AUGMENTATED_PATH + '/Rotated.png', dpi = 200,
#                       papertype = 'a5') 
#
#
#Image_Resize = Pre_Process_Re_Size(Inputs)
#
## 5. Show Results
#
#ix = random.randint(0, len(Inputs))
#
#img = Inputs[ix]
#mask = Ground_Truth [ix]
#resized = Image_Resize [ix]
#gray = Gray_Scale[ix]
#
#print('Input Image')
#imshow(img)
#plt.show()
#
#print('Mask')
#imshow(mask[:,:,1])
#plt.show()
#
#print('Resized Image')
#imshow(resized)
#plt.show()
#
#print('GrayScale Image')
#imshow(gray)
#plt.show()
#
#
#image_viewer = ImageViewer(gray)
#image_viewer.show()

In [ ]:
# 4.3. Function_3: Pre_Process_Re_Color():

Object_Pre_Preocessing = Pre_Processing_R2g(Inputs)
Gray_Scale = Pre_Processing_R2g.R_2_G(Object_Pre_Preocessing)


In [ ]:
# 4.4. Function_4: Pre_Process_Im_Saving():

New = 'D:/Python/Breast/'

def Pre_Process_Im_Saving(Path_Images, Path_Output, Tensor):
    
    for i, filename in enumerate(os.listdir(Path_Images)):
        
        imsave(fname='{}{}'.format(Path_Output, filename),
               arr=Tensor[i])
        
        print('{}: {}'.format(i, filename))
    
Pre_Process_Im_Saving(IMAGE_PATH, New, Gray_Scale)